

*  This tutorial aims to predict immunogenicity of peptides derived from glioblastoma patients using self-antigen model
*  All data and models can be retrieved from https://drive.google.com/drive/folders/15A2P5xP2c-q48vVGPRB7h7uHEMycPYoX?usp=drive_link 




In [ ]:
!pip install -q transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sys
import csv
import random
import os as os
from io import StringIO
import keras
from tensorflow.keras import layers
from keras.layers import Input,Dense,concatenate,Dropout,AveragePooling2D
from keras.models import Model,load_model                                                      
from keras import backend as K
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import auc,precision_recall_curve,roc_curve,confusion_matrix
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding, Input, LSTM, RNN, SimpleRNN, Bidirectional, Concatenate, GlobalMaxPool1D, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Conv2D
from keras.utils.vis_utils import plot_model
import transformers
from transformers import T5Tokenizer, T5Model, BertModel, BertForMaskedLM, BertTokenizer, pipeline, BertConfig, EncoderDecoderConfig, EncoderDecoderModel
from transformers import TFBertModel, TFXLNetModel, XLNetTokenizer,XLNetConfig, TFT5EncoderModel
import re
import pickle
import seaborn as sb
import matplotlib.pyplot as plt
from keras_preprocessing.sequence import pad_sequences

print('transformers version: %s' %transformers.__version__)
print('tensorflow version: %s' %tf.__version__)
print('keras version: %s' %keras.__version__)
print(sys.version)

transformers version: 4.25.1
tensorflow version: 2.9.2
keras version: 2.9.0
3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]


### Auxiliary functions

In [ ]:
# plotting scripts 
from tensorflow.keras.optimizers import Adam

# ROC
def draw_roc(bucket_roc):
    bucket = bucket_roc
    fig,ax = plt.subplots()
    for i in range(10):
        ax.plot(bucket[i][0],bucket[i][1],lw=0.5,label='CV(Fold={0}), AUC={1:.2f}'.format(i+1,bucket[i][3]))
    ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic')
    ax.legend(loc="lower right",fontsize=9)
    plt.show()

# PR
def draw_pr(bucket_pr):
    bucket = bucket_pr
    fig,ax = plt.subplots()
    for i in range(10):
        ax.plot(bucket[i][1],bucket[i][0],lw=0.5,label='CV(Fold={0}),AUC={1:.2f}'.format(i+1,bucket[i][3]))
    #baseline = np.sum(np.array(y_true) == 1) / len(y_true)  # 0.4735
    baseline = 0.4735
    ax.plot([0, 1], [baseline, baseline], color='navy', lw=2, linestyle='--')
    ax.set_xlim([0.0, 1.0])
    #ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
    ax.set_title('PR curve example')
    ax.legend(loc="lower left",fontsize=8)
    plt.show()


def draw_history(history):
    # history for accuracy 
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()


METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'),
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
]


def evaluate_roc_pr_10cv(test_model, X_train, y_train):
    # let's do a train/validation split
    bucket_roc = []
    bucket_pr = []
    for i in range(10):
        array = np.arange(len(X_train))
        train_index = np.random.choice(array,int(len(X_train)*0.9),replace=False)
        valid_index = [item for item in array if item not in train_index]

        input1_train = X_train[train_index]
        input1_valid = X_train[valid_index]
        label_train = y_train[train_index]
        label_valid = y_train[valid_index]

        model = test_model(X_train)
        callback_val = keras.callbacks.EarlyStopping(monitor='val_loss', patience=15,restore_best_weights=False)
        callback_train = keras.callbacks.EarlyStopping(monitor='loss',patience=2,restore_best_weights=False)
        history = model.fit(
            x=[input1_train],   # feed a list into
            y=label_train,
            validation_data = ([input1_valid],label_valid),
            batch_size=128,
            epochs=200,
            callbacks = [callback_val,callback_train])

        y_true = label_valid
        y_pred = model.predict([input1_valid])

        fpr,tpr,_ = roc_curve(y_true,y_pred)
        area = auc(fpr,tpr)
        bucket_roc.append((fpr,tpr,_,area))

        precision, recall, _ = precision_recall_curve(y_true, y_pred)
        area = auc(recall, precision)
        bucket_pr.append((precision, recall, _, area))

    draw_roc(bucket_roc)
    draw_pr(bucket_pr)

    
def evaluate_roc_pr_mlp_10cv(test_model, X_train, X_train_mlp, y_train):
    # let's do a train/validation split
    bucket_roc = []
    bucket_pr = []
    for i in range(10):
        array = np.arange(len(X_train))
        train_index = np.random.choice(array,int(len(X_train)*0.9),replace=False)
        valid_index = [item for item in array if item not in train_index]

        input1_train = X_train[train_index]
        input1_valid = X_train[valid_index]
        input1_mlp_train = X_train_mlp.iloc[train_index]
        input1_mlp_valid = X_train_mlp.iloc[valid_index]
        label_train = y_train[train_index]
        label_valid = y_train[valid_index]

        model = test_model(X_train)
        callback_val = keras.callbacks.EarlyStopping(monitor='val_loss', patience=15,restore_best_weights=False)
        callback_train = keras.callbacks.EarlyStopping(monitor='loss',patience=2,restore_best_weights=False)
        history = model.fit(
            x=[input1_train, input1_mlp_train],   # feed a list into
            y=label_train,
            validation_data = ([input1_valid, input1_mlp_valid],label_valid),
            batch_size=128,
            epochs=200,
            callbacks = [callback_val,callback_train])

        y_true = label_valid
        y_pred = model.predict([input1_valid, input1_mlp_valid])

        fpr,tpr,_ = roc_curve(y_true,y_pred)
        area = auc(fpr,tpr)
        bucket_roc.append((fpr,tpr,_,area))

        precision, recall, _ = precision_recall_curve(y_true, y_pred)
        area = auc(recall, precision)
        bucket_pr.append((precision, recall, _, area))

    draw_roc(bucket_roc)
    draw_pr(bucket_pr)


In [ ]:
# ROC
def draw_1roc(bucket_roc):
    bucket = bucket_roc
    fig,ax = plt.subplots()
    for i in range(1):
        ax.plot(bucket[i][0],bucket[i][1],lw=0.5,label='CV(Fold={0}), AUC={1:.2f}'.format(i+1,bucket[i][3]))
    ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic')
    ax.legend(loc="lower right",fontsize=9)
    plt.show()

# PR
def draw_1pr(bucket_pr):
    bucket = bucket_pr
    fig,ax = plt.subplots()
    for i in range(1):
        ax.plot(bucket[i][1],bucket[i][0],lw=0.5,label='CV(Fold={0}),AUC={1:.2f}'.format(i+1,bucket[i][3]))
    #baseline = np.sum(np.array(y_true) == 1) / len(y_true)  # 0.4735
    baseline = 0.4735
    ax.plot([0, 1], [baseline, baseline], color='navy', lw=2, linestyle='--')
    ax.set_xlim([0.0, 1.0])
    #ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
    ax.set_title('PR curve example')
    ax.legend(loc="lower left",fontsize=8)
    plt.show()


def evaluate_roc_pr_mlp_1cv(test_model, X_train, X_train_mlp, y_train):
    # let's do a train/validation split
    bucket_roc = []
    bucket_pr = []
    for i in range(1):
        array = np.arange(len(X_train))
        train_index = np.random.choice(array,int(len(X_train)*0.9),replace=False)
        valid_index = [item for item in array if item not in train_index]

        input1_train = X_train[train_index]
        input1_valid = X_train[valid_index]
        input1_mlp_train = X_train_mlp.iloc[train_index]
        input1_mlp_valid = X_train_mlp.iloc[valid_index]
        label_train = y_train[train_index]
        label_valid = y_train[valid_index]

        model = test_model(X_train)
        callback_val = keras.callbacks.EarlyStopping(monitor='val_loss', patience=15,restore_best_weights=False)
        callback_train = keras.callbacks.EarlyStopping(monitor='loss',patience=2,restore_best_weights=False)
        history = model.fit(
            x=[input1_train, input1_mlp_train],   # feed a list into
            y=label_train,
            validation_data = ([input1_valid, input1_mlp_valid],label_valid),
            batch_size=128,
            epochs=200,
            callbacks = [callback_val,callback_train])

        y_true = label_valid
        y_pred = model.predict([input1_valid, input1_mlp_valid])

        fpr,tpr,_ = roc_curve(y_true,y_pred)
        area = auc(fpr,tpr)
        bucket_roc.append((fpr,tpr,_,area))

        precision, recall, _ = precision_recall_curve(y_true, y_pred)
        area = auc(recall, precision)
        bucket_pr.append((precision, recall, _, area))

    draw_1roc(bucket_roc)
    draw_1pr(bucket_pr)


def evaluate_roc_pr_1cv(test_model, X_train, y_train):
    # let's do a train/validation split
    bucket_roc = []
    bucket_pr = []
    for i in range(1):
        array = np.arange(len(X_train))
        train_index = np.random.choice(array,int(len(X_train)*0.9),replace=False)
        valid_index = [item for item in array if item not in train_index]

        input1_train = X_train[train_index]
        input1_valid = X_train[valid_index]
        label_train = y_train[train_index]
        label_valid = y_train[valid_index]

        model = test_model(X_train)
        callback_val = keras.callbacks.EarlyStopping(monitor='val_loss', patience=15,restore_best_weights=False)
        callback_train = keras.callbacks.EarlyStopping(monitor='loss',patience=2,restore_best_weights=False)
        history = model.fit(
            x=[input1_train],   # feed a list into
            y=label_train,
            validation_data = ([input1_valid],label_valid),
            batch_size=128,
            epochs=200,
            callbacks = [callback_val,callback_train])

        y_true = label_valid
        y_pred = model.predict([input1_valid])

        fpr,tpr,_ = roc_curve(y_true,y_pred)
        area = auc(fpr,tpr)
        bucket_roc.append((fpr,tpr,_,area))

        precision, recall, _ = precision_recall_curve(y_true, y_pred)
        area = auc(recall, precision)
        bucket_pr.append((precision, recall, _, area))

    draw_1roc(bucket_roc)
    draw_1pr(bucket_pr)



## Data preparation

In [ ]:
def add_space_to_pep(peptides):
    peptide_space = [] 
    for ele in peptides:
        temp = [[]]
        for char in ele:
            temp.append([])
            temp[-1].append(char) 
        peptide_space.append(' '.join(''.join(ele) for ele in temp))
    peptide_space = [re.sub(r"[UZOB]", "X", sequence.lstrip()) for sequence in peptide_space]

    return peptide_space

In [ ]:
# training dataset 
sdata = pd.read_csv('data/selfantigen_data_forMODEL.csv')

sdata['Immunogenicity'] = sdata['Immunogenicity'].replace(['Positive', 'Positive-Low', 'Positive-Intermediate', 'Positive-High', 'Negative'], [1, 1,1,1,0])
y_train  = sdata['Immunogenicity'].values

In [ ]:
# test dataset
gbm_test = pd.read_csv('data/GBM_data_forTRAP.csv')
gbm_test.head()

,Peptide,n_hydrophobic,Immunogenicity,length,Hydrophobicity,MHC_Restriction,Rank,nlog2Rank,ContactPosition
0,FLEEIILKSL,3,Negative,10,0.300000,HLA-A*02:01,0.3277,1.609552,EEIILKS
1,FLRESQNPL,2,Negative,9,0.222222,HLA-A*02:01,0.5335,0.906440,RESQNP
2,GLALGTPLSI,4,Negative,10,0.400000,HLA-A*02:01,0.5356,0.900772,ALGTPLS
3,GLAVNLSQI,4,Negative,9,0.444444,HLA-A*02:01,0.5529,0.854910,AVNLSQ
4,GNLPDIEVRL,3,Negative,10,0.300000,HLA-A*02:01,0.5854,0.772505,LPDIEVR


In [ ]:
gbm_peptides = gbm_test.ContactPosition.values
gbm_input_peptides = add_space_to_pep(gbm_peptides)
gbm_test['Immunogenicity'] = gbm_test['Immunogenicity'].replace(['Positive', 'Positive-Low', 'Positive-Intermediate', 'Positive-High', 'Negative'], [1, 1,1,1,0])
y_test_gbm  = gbm_test['Immunogenicity'].values

## Embed sequence


*   Here, all peptides are post-padded seeing there isn't much difference between P3 padding and post-padding from previous analysis
*   Can do P3 padding - by instead of using tokenizer, do mid-padding as done in the previous script


In [ ]:
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False )
model = TFT5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5EncoderModel: ['decoder.block.14.layer.1.layer_norm.weight', 'decoder.block.23.layer.2.DenseReluDense.wi.weight', 'decoder.block.19.layer.1.EncDecAttention.o.weight', 'decoder.block.17.layer.0.SelfAttention.q.weight', 'decoder.block.9.layer.1.EncDecAttention.q.weight', 'decoder.block.16.layer.0.SelfAttention.o.weight', 'decoder.block.4.layer.1.EncDecAttention.v.weight', 'decoder.block.13.layer.0.SelfAttention.o.weight', 'decoder.block.3.layer.0.layer_norm.weight', 'decoder.block.21.layer.1.EncDecAttention.v.weight', 'decoder.block.0.layer.1.layer_norm.weight', 'decoder.block.21.layer.0.SelfAttention.v.weight', 'decoder.block.21.layer.0.SelfAttention.o.weight', 'decoder.block.6.layer.0.layer_norm.weight', 'decoder.block.4.layer.2.DenseReluDense.wi.weight', 'decoder.block.12.layer.0.SelfAttention.o.weight', 'decoder.block.18.layer.1.EncDecAttention.v.weight', 'decoder.block.17.layer.1.EncDecAttention.v

In [ ]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in gbm_input_peptides]
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts], padding="pre")

attention_masks = []
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

# Extracting sequences' features and load it into the CPU if needed
embedding = model(input_ids=input_ids)[0]
embedding = np.asarray(embedding)
pickle.dump( embedding, open( "output/gbm/protT5_xl_gbm_test.pkl", "wb" ) )

## Load peptide enbedded matrix

Same strategy was used to embed peptides from selfantigen_data_forMODEL dataset to prepare protT5_xl_peptides_1_train data

In [ ]:
# train dataset
protT5_xl_peptides_1_train = pickle.load( open( "output/bert_embedded_contacts_selfantigen/protT5_xl_peptides.pkl", "rb" ) )

# test dataset 
protT5_xl_peptides_1_gbm_test = pickle.load( open( "output/gbm/protT5_xl_gbm_test.pkl", "rb" ) )


## TRAP-1 model

The self-antigen TRAP-1 model was trained using peptides in selfantigen_data_forMODEL dataset

In [ ]:
# load pretrained softmax model 
self_trap = tf.keras.models.load_model('model/self_antigen_trap_softmax_model')

In [ ]:
# make prediction on test peptides
# this compute predictions 100 times with monte carlo dropouts 
bucket_test_DO = []
for i in range(100):
    peptides = gbm_test[['Peptide', 'nlog2Rank']]
    model_score = Model(self_trap.input, self_trap.get_layer('logits').output)
    X_test_DO = model_score.predict(x = [protT5_xl_peptides_1_gbm_test, gbm_test[['Hydrophobicity', 'nlog2Rank']]])
    X_test_DO = np.concatenate((X_test_DO, peptides),axis=1)
    bucket_test_DO.append(X_test_DO)

4/4 [==============================] - 0s 8ms/step


In [ ]:
# process output to compute average monte carlo dropout softmax probabilities 
test_DO_DT = pd.DataFrame(np.vstack((bucket_test_DO)))
test_DO_DT.columns = ["X0", "X1", "Peptide", 'nlog2Rank']
test_DO_DT = test_DO_DT.groupby(['Peptide', 'nlog2Rank']).agg(  #from here, become alphabetical 
    avg_X0 = pd.NamedAgg(column ='X0', aggfunc='mean'),
    avg_X1 = pd.NamedAgg(column ='X1', aggfunc='mean'))
test_DO_DT['maxprob_mean_dropout'] = test_DO_DT[['avg_X0', 'avg_X1']].max(axis=1)
test_DO_DT['TRAP'] = np.where((test_DO_DT['avg_X1'] >= test_DO_DT['avg_X0']), 'Positive', 'Negative')
test_DO_DT = test_DO_DT.reset_index()  
test_DO_DT.head()

,Peptide,nlog2Rank,avg_X0,avg_X1,maxprob_mean_dropout,TRAP
0,AAIESFVSV,1.493297,7.646901,-8.020171,7.646901,Negative
1,ALAGGLYEY,-0.722728,7.523490,-7.791161,7.523490,Negative
2,ALFCHQYDI,0.199090,-2.897632,2.792828,2.792828,Positive
3,ALGALLILQL,-0.725349,3.503604,-3.719930,3.503604,Negative
4,ALKSDFKLV,-0.480886,6.158325,-6.260597,6.158325,Negative


## OOD classifier

In [ ]:
from sklearn import linear_model
ood_data = pd.read_csv('data/ood_dropout_selfantigen.csv') # data for training OOD classifier 
X = ood_data[['maxprob_mean_dropout']]
y = ood_data.iloc[:,-1]
model = linear_model.LinearRegression().fit(X, y)

In [ ]:
# make prediciton on OOD classifier 
y_pred = model.predict(test_DO_DT[['maxprob_mean_dropout']])
ood_dt = pd.DataFrame(y_pred); ood_dt.columns = ['Confidence']

In [ ]:
trap_output = pd.concat([test_DO_DT.reset_index(drop=True), ood_dt.reset_index(drop=True)], axis=1)
trap_output = trap_output[["Peptide", 'nlog2Rank', "TRAP", 'maxprob_mean_dropout', 'Confidence']]
trap_output.head()
trap_output.to_csv('output/protT5_xl_gbm_test_prediction.csv')

# App demonstration on GBM data

In [ ]:
def parse_data(contents, filename):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV or TXT file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
        elif 'txt' or 'tsv' in filename:
            # Assume that the user upl, delimiter = r'\s+'oaded an excel file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')), delimiter = r'\s+')
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return df


def add_space_to_pep(peptides):
    peptide_space = [] 
    for ele in peptides:
        temp = [[]]
        for char in ele:
            temp.append([])
            temp[-1].append(char) 
        peptide_space.append(' '.join(''.join(ele) for ele in temp))
    peptide_space = [re.sub(r"[UZOB]", "X", sequence.lstrip()) for sequence in peptide_space]

    return peptide_space

def preprocess_test_peptides(test_data):

    # add contact positions 
    cont_peptides = []
    hydrophobicity = []
    for pep in test_data.Peptide:
      cont_pep = pep[2:-1]
      cont_peptides.append(cont_pep)
      hyd_counts = pep.count('A') + pep.count('V') + pep.count('L') + pep.count('M') + pep.count('W')
      length = len(pep)
      hydrophobicity.append(hyd_counts/length)

    cont_peptides = pd.DataFrame(cont_peptides); cont_peptides.columns = ['ContactPosition']
    hydrophobicity = pd.DataFrame(hydrophobicity); hydrophobicity.columns = ['Hydrophobicity']
    test_data = pd.concat([test_data, cont_peptides, hydrophobicity], axis=1)

    return test_data

def embed_test_peptides(test_data, tokenizer, TFT5EncoderModel):
    # embed test peptides 
    test_peptides = test_data.ContactPosition.values
    input_peptides = add_space_to_pep(test_peptides)
      
    tokenized_texts = [tokenizer.tokenize(sent) for sent in input_peptides]
    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts], padding="pre")

    attention_masks = []
    for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

    embedding = TFT5EncoderModel(input_ids=input_ids)[0]
    embedding = np.asarray(embedding)

    return embedding

############## THIS WORKING IN PROGRESS ##########################
def predict_trap(emebedding, test_data, trap, trap_ood):
    
  X_test = emebedding
  X_test_mlp = test_data[['Hydrophobicity', 'nlog2Rank']]

  # trap 
  y_pred = trap.predict([X_test, X_test_mlp])
  trap_pred = pd.DataFrame(y_pred); trap_pred.columns = ['TRAP']

  # trap-ood
  bucket_test_DO = []
  for i in range(100):
      peptides = test_data[['Peptide', 'nlog2Rank']]
      model_score = Model(trap.input, trap.get_layer('logits').output)
      X_test_DO = model_score.predict(x = [X_test, test_data[['Hydrophobicity', 'nlog2Rank']]])
      X_test_DO = np.concatenate((X_test_DO, peptides),axis=1)
      bucket_test_DO.append(X_test_DO)

  test_DO_DT = pd.DataFrame(np.vstack((bucket_test_DO)))
  test_DO_DT.columns = ["X0", "X1", "Peptide", 'nlog2Rank']
  test_DO_DT = test_DO_DT.groupby(['Peptide', 'nlog2Rank']).agg(  #from here, become alphabetical 
      avg_X0 = pd.NamedAgg(column ='X0', aggfunc='mean'),
      avg_X1 = pd.NamedAgg(column ='X1', aggfunc='mean'))
  test_DO_DT['maxprob_mean_dropout'] = test_DO_DT[['avg_X0', 'avg_X1']].max(axis=1)
  #test_DO_DT['TRAP'] = np.where((test_DO_DT['avg_X1'] >= test_DO_DT['avg_X0']), 'Positive', 'Negative')
  test_DO_DT = test_DO_DT.reset_index()  

  # ood classifier
  y_pred = trap_ood.predict(test_DO_DT[['maxprob_mean_dropout']])
  ood_dt = pd.DataFrame(y_pred); ood_dt.columns = ['Confidence']

  # process outputs
  trap_output = pd.concat([test_DO_DT.reset_index(drop=True), ood_dt.reset_index(drop=True)], axis=1)
    # need to left_join  
  trap_output = trap_output[["Peptide", 'nlog2Rank', "TRAP", 'maxprob_mean_dropout', 'Confidence']] 
  trap_output.rename(columns={"MCDropout": "maxprob_mean_dropout"})

  return trap_output


## Data loading 

In [ ]:
from sklearn import linear_model

######### tokenizer and embedding ###########
# tokenizer = T5Tokenizer.from_pretrained("rostlab/prot_t5_xl_uniref50", do_lower_case=False )
# TFT5EncoderModel = TFT5EncoderModel.from_pretrained("rostlab/prot_t5_xl_uniref50", from_pt=True)

########### OOD model #####################
# selfantigen model
self_trap = tf.keras.models.load_model('model/self_antigen_trap_model')
self_trap_softmax = tf.keras.models.load_model('model/selfantigen_trap_softmax_model')
ood_selfantigen_data = pd.read_csv('data/ood_dropout_selfantigen.csv')
self_trap_ood = linear_model.LinearRegression().fit(ood_selfantigen_data[['maxprob_mean_dropout']], ood_selfantigen_data.iloc[:,-1])

# pathogenic model 
path_trap = tf.keras.models.load_model('model/pathogenic_trap_model')
path_trap_softmax = tf.keras.models.load_model('model/pathogenic_trap_softmax_model')
ood_pathogenic_data = pd.read_csv('data/ood_dropout_pathogenic.csv')
path_trap_ood = linear_model.LinearRegression().fit(ood_pathogenic_data[['maxprob_mean_dropout']], ood_pathogenic_data.iloc[:,-1])

In [ ]:
# test dataset 
emebedding = pickle.load( open( "output/gbm/protT5_xl_gbm_test.pkl", "rb" ) )
gbm_data = pd.read_csv('data/GBM_data_forTRAP.csv')

## Data processing 

In [ ]:
###### INPUTS TO FUNCTION ###############
test_data = gbm_data
trap = self_trap
trap_softmax = self_trap_softmax
trap_ood = self_trap_ood

In [ ]:
###### FUNCTION ############### 
test_data = test_data[['Peptide', 'Hydrophobicity', 'nlog2Rank']].sort_values(['Peptide', 'nlog2Rank']).drop_duplicates().reset_index()
X_test_mlp = test_data[['Hydrophobicity', 'nlog2Rank']]

# do embedding here 

# trap 
y_pred = trap.predict([X_test, X_test_mlp])
trap_pred = pd.DataFrame(y_pred); trap_pred.columns = ['TRAP']
trap_pred = pd.concat([test_data[['Peptide', 'nlog2Rank']].reset_index(drop = True), trap_pred.reset_index(drop = True)], axis=1)  
    # output: peptide, nlog2, trap 

# trap-ood
bucket_test_DO = []
for i in range(100):
    peptides = test_data[['Peptide', 'nlog2Rank']]
    model_score = Model(trap_softmax.input, trap_softmax.get_layer('logits').output)
    X_test_DO = model_score.predict(x = [X_test, test_data[['Hydrophobicity', 'nlog2Rank']]])
    X_test_DO = np.concatenate((X_test_DO, peptides),axis=1)
    bucket_test_DO.append(X_test_DO)

test_DO_DT = pd.DataFrame(np.vstack((bucket_test_DO)))
test_DO_DT.columns = ["X0", "X1", "Peptide", 'nlog2Rank']
test_DO_DT = test_DO_DT.groupby(['Peptide', 'nlog2Rank']).agg(  #from here, become alphabetical 
    avg_X0 = pd.NamedAgg(column ='X0', aggfunc='mean'),
    avg_X1 = pd.NamedAgg(column ='X1', aggfunc='mean'))
test_DO_DT['maxprob_mean_dropout'] = test_DO_DT[['avg_X0', 'avg_X1']].max(axis=1)
#test_DO_DT['TRAP'] = np.where((test_DO_DT['avg_X1'] >= test_DO_DT['avg_X0']), 'Positive', 'Negative')
test_DO_DT = test_DO_DT.reset_index()  

# ood classifier
y_pred = trap_ood.predict(test_DO_DT[['maxprob_mean_dropout']])
ood_dt = pd.DataFrame(y_pred); ood_dt.columns = ['Confidence']
ood_dt = pd.concat([test_DO_DT[["Peptide", 'nlog2Rank',  'maxprob_mean_dropout']].reset_index(drop=True), 
                    ood_dt.reset_index(drop=True)], axis=1) 
  # output: peptide, nlog2, maxprob_mean_dropout, confidence 

# merge outputs 
trap_pred['nlog2Rank']=trap_pred['nlog2Rank'].apply(lambda x:round(x,3))
ood_dt['nlog2Rank']=ood_dt['nlog2Rank'].apply(lambda x:round(x,3))
trap_output = pd.merge(trap_pred, ood_dt,  how='left', left_on=['Peptide','nlog2Rank'], right_on = ['Peptide','nlog2Rank'])
trap_output.rename(columns={'maxprob_mean_dropout': 'MCDropout'}, inplace=True)


In [ ]:
#trap_output.rename(columns={"MCDropout": "maxprob_mean_dropout"})
trap_output.rename(columns={'maxprob_mean_dropout': 'MCDropout'}, inplace=True)
trap_output.head()

,Peptide,nlog2Rank,TRAP,MCDropout,Confidence
0,AAIESFVSV,1.493,0.001471,5.757001,0.846314
1,ALAGGLYEY,-0.723,0.272071,1.210100,0.709710
2,ALFCHQYDI,0.199,0.030533,1.176501,0.708700
3,ALGALLILQL,-0.725,0.001783,5.379736,0.834980
4,ALKSDFKLV,-0.481,0.000014,8.076635,0.916004


In [ ]:
import plotly.express as px
# plot 
prediction = trap_output.drop_duplicates()
if trap_output.shape[0] > 40: prediction_dt = prediction.nlargest(40,'TRAP')
else: prediction_dt = prediction.nlargest(trap_output.shape[0],'TRAP')
fig_output = px.bar(prediction_dt, 
              x="Peptide", 
              y="TRAP", 
              color="Confidence", color_continuous_scale='sunsetdark',
              width=800, height=400, template="simple_white").add_shape( # add a horizontal "target" line
          type="line", line_color="salmon", line_width=3, opacity=1, line_dash="dot",
          x0=0, x1=1, xref="paper", y0=0.5, y1=0.5, yref="y")
fig_output.update_layout(transition_duration=500)

In [ ]:
trap_output.head()

,Peptide,nlog2Rank,TRAP,maxprob_mean_dropout,Confidence
0,AAIESFVSV,1.493,0.001471,5.757001,0.846314
1,ALAGGLYEY,-0.723,0.272071,1.210100,0.709710
2,ALFCHQYDI,0.199,0.030533,1.176501,0.708700
3,ALGALLILQL,-0.725,0.001783,5.379736,0.834980
4,ALKSDFKLV,-0.481,0.000014,8.076635,0.916004
